<a href="https://colab.research.google.com/github/DrAlexSanz/NLP-SPEC-C2/blob/master/W1/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task

*   Get a word count given a corpus
*   Get a word probability in the corpus
*   Manipulate strings
*   Filter strings
*   Implement Minimum edit distance to compare strings and to help find the     optimal path for the edits.
*   Understand how dynamic programming works



### Edit Distance
In this assignment, you will implement models that correct words that are 1 and 2 edit distances away.

We say two words are n edit distance away from each other when we need n edits to change one word into another.
An edit could consist of one of the following options:

Delete (remove a letter): ‘hat’ => ‘at, ha, ht’
Switch (swap 2 adjacent letters): ‘eta’ => ‘eat, tea,...’
Replace (change 1 letter to another): ‘jat’ => ‘hat, rat, cat, mat, ...’
Insert (add a letter): ‘te’ => ‘the, ten, ate, ...’

In [1]:
# Let's start by importing all I will need and download the Shakespeare file

import re
from collections import Counter
import numpy as np
import pandas as pd

!wget https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C2/master/W1/shakespeare.txt

--2020-10-05 05:49:00--  https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C2/master/W1/shakespeare.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306996 (300K) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>] 299.80K  --.-KB/s    in 0.06s   

2020-10-05 05:49:01 (4.98 MB/s) - ‘shakespeare.txt’ saved [306996/306996]



### Exercise 1
Implement the function process_data which

1) Reads in a corpus (text file)

2) Changes everything to lowercase

3) Returns a list of words.

Options and Hints
If you would like more of a real-life practice, don't open the 'Hints' below (yet) and try searching the web to derive your answer.
If you want a little help, click on the green "General Hints" section by clicking on it with your mouse.
If you get stuck or are not getting the expected results, click on the green 'Detailed Hints' section to get hints for each step that you'll take to complete this function.

In [2]:
def process_data():
    """
    Read shakespeare file
    Make it lowercase
    Return the list of words
    I won't pass the path or the file as an argument this time
    """

    with open("shakespeare.txt") as f:
        data = f.read()

    data = data.lower()
    words = re.findall(r"\w+", data) # If I put (\W+) It will also return the spaces as characters
    # I could probably use .split()

    return words



In [3]:
words = process_data()

# print(words)

Now I convert the whole list to a set to avoid duplicates

In [4]:
print("Total words in file:", len(words))
vocab = set(words)
print("Unique words in file:", len(vocab))

Total words in file: 53614
Unique words in file: 6116


## Exercise 2
Implement a get_count function that returns a dictionary

The dictionary's keys are words. The value for each word is the number of times that word appears in the corpus.

* Try implementing this using a for loop and a regular dictionary. This may be good practice for similar coding interview questions
* You can also use defaultdict instead of a regualr dictionary, along with the for loop
* Otherwise, to skip using a for loop, you can use Python's Counter class

In [5]:
def get_count_auto(word_list):
    """
    Implement a get_count function that returns a dictionary.
    The dictionary's keys are words.
    The value for each word is the number of times that word appears in the corpus.
    """

    word_count_dict = Counter(word_list)

    return word_count_dict

def get_count_manual(word_list):
    """
    Implement a get_count function that returns a dictionary.
    The dictionary's keys are words.
    The value for each word is the number of times that word appears in the corpus.
    """

    word_count_dict = {}

    for i in word_list:
        word_count_dict[i] = word_count_dict.get(i, 0) + 1

    return word_count_dict

In [6]:
auto_voc = get_count_auto(words)
man_voc= get_count_manual(words)

print(list(auto_voc.items())[:10])
print(list(man_voc.items())[:10])

[('o', 157), ('for', 474), ('a', 757), ('muse', 18), ('of', 1094), ('fire', 22), ('that', 785), ('would', 138), ('ascend', 1), ('the', 1525)]
[('o', 157), ('for', 474), ('a', 757), ('muse', 18), ('of', 1094), ('fire', 22), ('that', 785), ('would', 138), ('ascend', 1), ('the', 1525)]


### Exercise 3

Given the dictionary of word counts, compute the probability that each word will appear if randomly selected from the corpus of words. Remember:

$P(w) = \frac{Count(w)}{Total(w)}$ 

In [7]:
def get_probs(voc):
    """
    Get a dictionary with the vocabulary and calculate the probability of a word
    using the count
    """

    voc_prob = {}
    total_words = sum(voc.values())
    for w in voc.keys():
      voc_prob[w] = voc.get(w, 0)/total_words

    return voc_prob

In [8]:
probs = get_probs(auto_voc)

print(len(probs))
print(sum(probs.values()))

6116
0.999999999999934


##Part 2: String Manipulations
Now, that you have computed $P(w_i)$ for all the words in the corpus, you will write a few functions to manipulate strings so that you can edit the erroneous strings and return the right spellings of the words. In this section, you will implement four functions:

*delete_letter*: given a word, it returns all the possible strings that have one character removed.
*switch_letter*: given a word, it returns all the possible strings that have two adjacent letters switched.
*replace_letter*: given a word, it returns all the possible strings that have one character replaced by another different letter.
*insert_letter*: given a word, it returns all the possible strings that have an additional character inserted.

### Exercise 4
Instructions for delete_letter(): Implement a delete_letter() function that, given a word, returns a list of strings with one character deleted.

For example, given the word nice, it would return the set: {'ice', 'nce', 'nic', 'nie'}.

**Step 1:** Create a list of 'splits'. This is all the ways you can split a word into Left and Right: For example,
'nice is split into : [('', 'nice'), ('n', 'ice'), ('ni', 'ce'), ('nic', 'e'), ('nice', '')] This is common to all four functions (delete, replace, switch, insert).
**Step 2:** This is specific to delete_letter. Here, we are generating all words that result from deleting one character.
This can be done in a single line with a list comprehension. You can makes use of this type of syntax:
[f(a,b) for a, b in splits if condition]

For our 'nice' example you get: ['ice', 'nce', 'nie', 'nic'].

In [9]:
def delete_letter(word):
    """
    take one word, split it into L and R substrings. Then delete 
    the first letter of R and combine the result.
    """

    splits = [(word[:i], word[i:]) for i in range(len(word))]
    results = [(L + R[1:]) for L, R in splits]

    return results

In [10]:
cip = delete_letter("cipote")
print(cip) #Check that it doesn't return the full word (range(len(w) +1))

['ipote', 'cpote', 'ciote', 'cipte', 'cipoe', 'cipot']


## Exercise 5
Instructions for switch_letter(): Now implement a function that switches two letters in a word. It takes in a word and returns a list of all the possible switches of two letters that are adjacent to each other. It shouldn't return the original word.

For example, given the word 'eta', it returns {'eat', 'tea'}, but does not return 'ate'.
Step 1: is the same as in delete_letter()
Step 2: A list comprehension or for loop which forms strings by swapping adjacent letters. This is of the form:
[f(L,R) for L, R in splits if condition] where 'condition' will test the length of R in a given iteration. See below.

In [11]:
def switch_letter(word):
    """
    Get a word and produce all the combinations of two adjacent letters swapped
    It shouldn't return the original word
    """

    splits = [(word[:i], word[i:]) for i in range(len(word)) if (len(word[:i]) and len(word[:i])) > 0] # Same as before

    swaps = [L[:-1] + R[0] + L[-1] + R[1:] for L, R in splits]

    return swaps

In [12]:
test = switch_letter("Cipote")
print(test)

['iCpote', 'Cpiote', 'Ciopte', 'Ciptoe', 'Cipoet']


### Exercise 6
Instructions for replace_letter(): Now implement a function that takes in a word and returns a list of strings with one replaced letter from the original word.

Step 1: is the same as in delete_letter()

Step 2: A list comprehension or for loop which form strings by replacing letters. This can be of the form:
[f(a,b,c) for a, b in splits if condition for c in string]
**Note the use of the second for loop.**

It is expected in this routine that one or more of the replacements will include the original word. For example, replacing the first letter of 'ear' with 'e' will return 'ear'.

Step 3: Remove the original input letter from the output.

* To remove a word from a list, first store its contents inside a set()
* Use set.discard('the_word') to remove a word in a set (if the word does not exist in the set, then it will not throw a KeyError.
* Using set.remove('the_word') throws a KeyError if the word does not exist in the set.

In [13]:
def replace_letter(word):
    """
    Split, replace, and then remove the original
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'

    splits = [(word[:i], word[i:]) for i in range(len(word))] # Same as before

    replaced = [L + alph + (R[1:] if len(R) > 1 else "") for L, R in splits for alph in letters]

    # Make replaced a set to get only uniques

    replaced_set = set(replaced)
    replaced_set.remove(word)
    results = sorted(list(replaced_set))

    return results



In [14]:
a = replace_letter("cipote") # Test with aa in case of doubt. It works
print(len(a))


150


### Exercise 7
Instructions for insert_letter(): Now implement a function that takes in a word and returns a list with a letter inserted at every offset.

* Step 1: is the same as in delete_letter()

* Step 2: This can be a list comprehension of the form:
[f(a,b,c) for a, b in splits if condition for c in string]

In [15]:
def insert_letter(word):

    letters = 'abcdefghijklmnopqrstuvwxyz'

    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]

    inserts = [L + alph + R for L, R in splits for alph in letters]


    return inserts


In [16]:
a = insert_letter("cipote")
print(len(a)) # Should be 78 with input = "at"

182


# Part 3: Combining the edits
Now that you have implemented the string manipulations, you will create two functions that, given a string, will return all the possible single and double edits on that string. These will be edit_one_letter() and edit_two_letters().

### 3.1 Edit one letter

### Exercise 8
Instructions: Implement the edit_one_letter function to get all the possible edits that are one edit away from a word. The edits consist of the replace, insert, delete, and optionally the switch operation. You should use the previous functions you have already implemented to complete this function. The 'switch' function is a less common edit function, so its use will be selected by an "allow_switches" input argument.

* Note that those functions return lists while this function should return a python set. Utilizing a set eliminates any duplicate entries.

* Each of the functions returns a list. You can combine lists using the `+` operator.
* To get unique strings (avoid duplicates), you can use the set() function.

In [17]:
def edit_one_leter(word, allow_switch = True):

    deletes = delete_letter(word)

    replaces = replace_letter(word)

    inserts = insert_letter(word)

    if allow_switch:
        switches = switch_letter(word)
    
    else:
        switches = []

    outputs = deletes + replaces + inserts + switches

    result = set(outputs)

    return result

In [19]:
test = edit_one_leter("cipote") # at gives len = 129 as expected

print(len(test))

337


# Part 3.2 Edit two letters

### Exercise 9
Now you can generalize this to implement to get two edits on a word. To do so, you would have to get all the possible edits on a single word and then for each modified word, you would have to modify it again.

**Instructions:** Implement the edit_two_letters function that returns a set of words that are two edits away. Note that creating additional edits based on the edit_one_letter function may 'restore' some one_edits to zero or one edits. That is allowed here. This accounted for in get_corrections.

* You will likely want to take the union of two sets.
* You can either use set.union() or use the '|' (or operator) to union two sets
* See the documentation Python sets for examples of using operators or functions of the Python set.

In [38]:
def edit_two_letters(word, allow_switch = True):

    second_edits_set = set()
    ed_one = edit_one_leter(word)

    # second_edits = [edit_one_leter(w) for w in ed_one if w]

    for w in ed_one:

        if w:
            second_edits = edit_one_leter(w)
            second_edits_set.update(second_edits)

    # result = set(second_edits)

    return second_edits_set

In [40]:
test = edit_two_letters("cipote") # "a" produces 2564

print(len(test))

52170


##Part 3-3: suggest spelling suggestions
Now you will use your edit_two_letters function to get a set of all the possible 2 edits on your word. You will then use those strings to get the most probable word you meant to type aka your typing suggestion.


### Exercise 10
Instructions: Implement get_corrections, which returns a list of zero to n possible suggestion tuples of the form (word, probability_of_word).

### Step 1: Generate suggestions for a supplied word: You'll use the edit functions you have developed. The 'suggestion algorithm' should follow this logic:

If the word is in the vocabulary, suggest the word.
Otherwise, if there are suggestions from edit_one_letter that are in the vocabulary, use those.
Otherwise, if there are suggestions from edit_two_letters that are in the vocabulary, use those.
Otherwise, suggest the input word.*
The idea is that words generated from fewer edits are more likely than words with more edits.
Note:

Edits of one or two letters may 'restore' strings to either zero or one edit. This algorithm accounts for this by preferentially selecting lower distance edits first.

In [49]:
def get_corrections(word, probs, vocab, n):
    """
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    
    """

    suggestions = []
    n_best = []

    suggestions = list((word in vocab and word) or edit_one_leter(word).intersection(vocab) or edit_two_letters(word).intersection(vocab))

    n_best = [[s, probs[s]] for s in list(reversed(suggestions))]
    n_best = n_best[:n]

    return n_best

In [51]:
test = get_corrections("cipot", probs, vocab, 13)
print(test)

[['capet', 5.595553400231283e-05], ['pilot', 1.865184466743761e-05], ['chopt', 1.865184466743761e-05], ['spot', 1.865184466743761e-05], ['riot', 1.865184466743761e-05]]
